# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Observables.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Observables

This notebook contains some useful shared tools to work with [RX Observables](https://rxpy.readthedocs.io/en/latest/reference_observable.html).


In [ ]:
import logging
import rx
import rx.operators as ops
import typing


# printing_observer function

This function returns an object that can subscribe to an `Observable` and print out each item.

In [ ]:
def printing_observer() -> rx.core.typing.Observer:
    class PrintingObserverSubscriber(rx.core.typing.Observer):
        def __init__(self) -> None:
            self.something_output = False

        def on_next(self, item: typing.Any) -> None:
            self.something_output = True
            print(item)

        def on_error(self, ex: Exception) -> None:
            self.something_output = True
            print(ex)

        def on_completed(self) -> None:
            if not self.something_output:
                print("No items to show.")

    return PrintingObserverSubscriber()


# CollectingObserverSubscriber class

This class can subscribe to an `Observable` and collect each item.

In [ ]:
class CollectingObserverSubscriber(rx.core.typing.Observer):
    def __init__(self) -> None:
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.collected: typing.List[typing.Any] = []

    def on_next(self, item: typing.Any) -> None:
        self.collected += [item]

    def on_error(self, ex: Exception) -> None:
        self.logger.error(f"Received error: {ex}")

    def on_completed(self) -> None:
        pass


## debug_print_item function

This is a handy item that can be added to a pipeline to show what is being passed at that particular stage. For example, this shows how to print the item before and after filtering:
```
fetch().pipe(
    ops.map(debug_print_item("Unfiltered:")),
    ops.filter(lambda item: item.something is not None),
    ops.map(debug_print_item("Filtered:")),
    ops.filter(lambda item: item.something_else()),
    ops.map(act_on_item)
).subscribe(some_subscriber)
```

In [ ]:
def debug_print_item(title: str) -> typing.Callable[[typing.Any], typing.Any]:
    def _debug_print_item(item: typing.Any) -> typing.Any:
        print(title, item)
        return item
    return _debug_print_item


# 🏃 Running

A few quick examples to show how to use these functions

In [ ]:
if __name__ == "__main__":
    rx.from_([1, 2, 3, 4, 5]).subscribe(printing_observer())
    rx.from_([1, 2, 3, 4, 5]).pipe(
        ops.filter(lambda item: (item % 2) == 0),
    ).subscribe(printing_observer())

    collector = CollectingObserverSubscriber()
    rx.from_(["a", "b", "c"]).subscribe(collector)
    print(collector.collected)

    rx.from_([1, 2, 3, 4, 5]).pipe(
        ops.map(debug_print_item("Before even check:")),
        ops.filter(lambda item: (item % 2) == 0),
        ops.map(debug_print_item("After even check:")),
    ).subscribe(printing_observer())
